# Circuits Statistics
The objective of this notebook is to analize the data that's being used in our database in order to see which statistics can be utilised.  
Lets start by importing the pandas library.

Adding to git repo

While trying to read the csv file, I received the following error:  
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xcc in position 4: invalid continuation byte  

So the csv file is not compatible with the normal UTF-8 encoding.  
I found the following information on the issue: [Unicode Error](https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python)  
By using ISO-8859-1 endoding instead of UTF-8, I was able to import the csv file as a pandas dataframe.

In [1]:
import pandas as pd

fileData = 'circuits.csv'
df  = pd.read_csv(fileData, header=0, index_col=None, encoding = "ISO-8859-1")
df

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,NaN,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,NaN,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,MontmelÌ_,Spain,41.57000,2.26111,NaN,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,NaN,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...,...,...,...,...,...,...,...
68,69,americas,Circuit of the Americas,Austin,USA,30.13280,-97.64110,NaN,http://en.wikipedia.org/wiki/Circuit_of_the_Am...
69,70,red_bull_ring,Red Bull Ring,Spielburg,Austria,47.21970,14.76470,NaN,http://en.wikipedia.org/wiki/Red_Bull_Ring
70,71,sochi,Sochi Autodrom,Sochi,Russia,43.40570,39.95780,NaN,http://en.wikipedia.org/wiki/Sochi_Autodrom
71,72,port_imperial,Port Imperial Street Circuit,New Jersey,USA,40.77690,-74.01110,NaN,http://en.wikipedia.org/wiki/Port_Imperial_Str...


Now that we've been able to read the .csv file, lets have a look at the dataframe and see what we can do with the data.  
Something that would be nice would be to see the locations of the circuits on an interactive map. We can use the latitude and longitude co-ordinates.

In [2]:
df.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,NaN,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,NaN,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,MontmelÌ_,Spain,41.57000,2.26111,NaN,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,NaN,http://en.wikipedia.org/wiki/Istanbul_Park


One library we can use for interactive maps is the ipyleaflet library.  
Reference: [Github - ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet)  

We also need to install the library in order to use it.  
Installation guide: [Install ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/installation.html#using-conda)  

After following the installation steps, the map did not render and instead showed the object as text.  
I found the following solution to the problem, which requires you to also install the labextentions.  
Solution: [Failed to display map](https://github.com/jupyter-widgets/ipyleaflet/issues/173)

In [3]:
from ipyleaflet import Map, basemaps

# Map centred on (60 degrees latitude et -2.2 degrees longitude)
# Latitude, longitude
Map(center = (25, 10), zoom = 2, min_zoom = 1, max_zoom = 20, basemap=basemaps.Stamen.Terrain)

Map(center=[25, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

Now that we have a working map, lets see if we can display the locations by adding markers to the map, based on the lattitude and longitude co-ordinates from our dataframe.  
Reference:  [Markers - ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/api_reference/marker.html)  

It would also be great to add popups to the markers, so that when clicked, gives information about the circuit.  
Reference: [Popups](https://ipyleaflet.readthedocs.io/en/latest/api_reference/popup.html)

In [4]:
from ipyleaflet import Marker, Popup
from ipywidgets import HTML

m = Map(center = (25, 10), zoom = 2, min_zoom = 2, max_zoom = 30, basemap=basemaps.CartoDB.Positron)

for name, country, city, lat, lng in zip(df['name'], df['country'], df['location'], df['lat'], df['lng']):
    circuitPosition = (lat, lng)
    #print(circuitPosition)
    marker = Marker(location=circuitPosition, title=name, draggable=False, opacity=0.5, rise_on_hover=True)
    m.add_layer(marker);
    
    message = HTML()
    message.value = "<b>Name: </b>" + name + "</br>" + "<b>Country: </b>" + country + "</br>" + "<b>City: </b>" + city
    
    # Popup with a given location on the map:
    popup = Popup(
        location=circuitPosition,
        child=message,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
    m.add_layer(popup)

    # Popup associated to a layer
    marker.popup = message

m

Map(center=[25, 10], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

## Conclusion
Other than displaying the circuit locations, not much more can be done with this data in isolation.  
I'll need to use it with other data sets to see if more data can be combined to extract more information from the avaialable datasets.